In [117]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import math
import imageio.v2 as imageio
import SimpleITK as sitk
from radiomics import featureextractor
import pandas as pd
import re
import numpy as np
from radiomics import glszm
from io import BytesIO
from numpy import ones, kron, mean, eye, hstack, dot, tile
from numpy.linalg import pinv
from matplotlib.image import imread

In [119]:
# change path
save_path = '/projects/YG'

os.getcwd()
os.chdir(save_path)

In [120]:
invariant_feature_folder = os.path.join(save_path, "Invariant_Features")
os.makedirs(invariant_feature_folder, exist_ok=True) 

# inphase & outphase 

In [22]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from radiomics import featureextractor

extractor = featureextractor.RadiomicsFeatureExtractor()

def process_phase_images(save_path, phase_type='inphase'):
    Ng_values = [64, 96, 128, 160, 192, 224, 256]
    radiomic_features_by_ng = {}
    for Ng in Ng_values:
        print(f"Processing {phase_type} images at Ng = {Ng}...")
        radiomic_features = []

        for patient_folder in os.listdir(save_path):
            patient_path = os.path.join(save_path, patient_folder)
            if not os.path.isdir(patient_path):
                continue  
            npz_files = [f for f in os.listdir(patient_path) if f.endswith("_data.npz")]
            if not npz_files:
                print(f"Skipping {patient_folder}: No NPZ data file found.")
                continue
            npz_file_path = os.path.join(patient_path, npz_files[0])
            data = np.load(npz_file_path)
            image = data[phase_type] 
            segmentation_image = data["segmentation"]
            quantized_image = np.round((image / np.max(image)) * (Ng - 1)).astype(np.uint8)
            quantized_npz_path = os.path.join(patient_path, f"{patient_folder}_quantized_{phase_type}_Ng_{Ng}.npz")
            np.savez(quantized_npz_path, quantized_image=quantized_image, segmentation=segmentation_image)
            image_sitk = sitk.GetImageFromArray(quantized_image)
            segmentation_sitk = sitk.GetImageFromArray((segmentation_image > 0).astype(np.uint8))

            result = extractor.execute(image_sitk, segmentation_sitk)

            filtered_result = {key: value for key, value in result.items() if "diagnostics" not in key}
            radiomic_features.append({"PatientID": patient_folder, **filtered_result})

        radiomic_features_df = pd.DataFrame(radiomic_features)

        radiomic_features_df = radiomic_features_df[
            radiomic_features_df.columns.drop(list(radiomic_features_df.filter(regex='firstorder')))
        ]
        radiomic_features_df = radiomic_features_df[
            radiomic_features_df.columns.drop(list(radiomic_features_df.filter(regex='diagnostics')))
        ]
        radiomic_features_df = radiomic_features_df.sort_index()

        radiomic_features_by_ng[Ng] = radiomic_features_df

    consolidated_npz_path = os.path.join(save_path, f"radiomic_features_by_ng_{phase_type}.npz")
    np.savez(consolidated_npz_path, **{f"Ng_{Ng}": df.to_dict(orient='list') for Ng, df in radiomic_features_by_ng.items()})
    print(f"Saved consolidated radiomic features for {phase_type} to {consolidated_npz_path}")

In [ ]:
inphase_data = np.load(f"{save_path}/radiomic_features_by_ng_inphase.npz", allow_pickle=True)
ng_x_inphase_features = pd.DataFrame(inphase_data["Ng_128"].item())
ng_x_inphase_features

In [ ]:
invariant_feature_folder = os.path.join(save_path, "Invariant_Features")
consolidated_excel_path = os.path.join(invariant_feature_folder, "outphase_Ng_64-256.xlsx")

npz_path = os.path.join(save_path, "radiomic_features_by_ng_outphase.npz")
radiomic_features_data = np.load(npz_path, allow_pickle=True)

excel_writer = pd.ExcelWriter(consolidated_excel_path, engine="xlsxwriter")
for Ng_key in radiomic_features_data.files:
    features_dict = radiomic_features_data[Ng_key].item() 
    features_df = pd.DataFrame(features_dict)

    if not features_df.empty:
        features_df.to_excel(excel_writer, sheet_name=Ng_key, index=False)
    else:
        print(f"Skipping {Ng_key} because the DataFrame is empty.")

excel_writer.close()

In [ ]:
consolidated_excel_path = os.path.join("/projects/YG", "inphase_Ng_64-256_75features.xlsx")
sheet_name = "Ng_128" 
df = pd.read_excel(consolidated_excel_path, sheet_name=sheet_name)
df

## feature vs. Ng

In [43]:
# phase_type = 'outphase'
phase_type = 'inphase'
diff_Ngs_file = pd.ExcelFile(f'./Invariant_Features/{phase_type}_Ng_64-256_75features.xlsx')
excel_writer2 = pd.ExcelWriter(f'./Invariant_Features/{phase_type}_feature_vs_Ng.xlsx', engine='xlsxwriter')

first_sheet_name = diff_Ngs_file.sheet_names[0] 
first_df = diff_Ngs_file.parse(first_sheet_name)
num_cols = len(first_df.columns)

for col_index in range(1,num_cols):
    column_data = []

    for sheet_name in diff_Ngs_file.sheet_names:
        df = diff_Ngs_file.parse(sheet_name)
        column = df.iloc[:, col_index]
        column_data.append(column)

    column_data_df = pd.DataFrame(column_data).T
    column_headers = [64, 96, 128, 160, 192, 224, 256]
    column_data_df.columns = column_headers

    name = "_".join(first_df.columns[col_index].split("_")[1:]) 
    column_data_df.to_excel(excel_writer2, sheet_name=name[:31], index=False)

excel_writer2.close()

In [ ]:
phase_type = 'inphase'

excel_path = os.path.join(invariant_feature_folder, f"{phase_type}_feature_vs_Ng.xlsx")

excel_file = pd.ExcelFile(excel_path)
sheet_names = excel_file.sheet_names
chosen_sheet = "glcm_Autocorrelation"
print(f"Loading data from sheet: {chosen_sheet}")
df = excel_file.parse(sheet_name=chosen_sheet)
df

In [ ]:
consolidated_excel_path = os.path.join("/projects/YG", "inphase_Ng_64-256_75features.xlsx")
sheet_name = "Ng_128" 
df = pd.read_excel(consolidated_excel_path, sheet_name=sheet_name)

print(f"Loaded data from sheet: {sheet_name}")
df

## Find invariant feature

In [13]:
def icc(Y, icc_type='ICC(3,1)'):

    [n, k] = Y.shape 
    dfc = k - 1
    dfe = (n - 1) * (k-1)
    dfr = n - 1
    mean_Y = np.mean(Y)
    SST = ((Y - mean_Y) ** 2).sum()

    x = np.kron(np.eye(k), np.ones((n, 1)))  # sessions
    x0 = np.tile(np.eye(n), (k, 1))  # subjects
    X = np.hstack([x, x0])

    predicted_Y = np.dot(np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))),
                                X.T), Y.flatten('F'))
    residuals = Y.flatten('F') - predicted_Y
    SSE = (residuals ** 2).sum()

    MSE = SSE / dfe

    SSC = ((np.mean(Y, 0) - mean_Y) ** 2).sum() * n
    MSC = SSC / dfc  
    SSR = SST - SSC - SSE
    MSR = SSR / dfr

    if icc_type == 'icc1':
        NotImplementedError("This method isn't implemented yet.")

    elif icc_type == 'ICC(2,1)' or icc_type == 'ICC(2,k)':
        if icc_type == 'ICC(2,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE + k * (MSC - MSE) / n)

    elif icc_type == 'ICC(3,1)' or icc_type == 'ICC(3,k)':
        if icc_type == 'ICC(3,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE)

    return ICC

In [ ]:
features_file = pd.ExcelFile('./Invariant_Features/outphase_feature_vs_Ng.xlsx')
df_non_norm = features_file.parse('ngtdm_Strength')
df_non_norm.shape

In [ ]:
df_non_norm.columns

In [ ]:
df_norm = pd.DataFrame(columns=df_non_norm.columns)
for col in df_non_norm.columns:
    header_value = df_non_norm[col].name

    df_norm[col] = df_non_norm[col]*(math.log(header_value)) 
df_norm.head()

In [ ]:
array = df_non_norm.values
ICC_tmp = icc(array)
ICC_tmp = round(ICC_tmp, 2)
print("Non-normalization ICC:", ICC_tmp)

array2 = df_norm.values
ICC_tmp2 = icc(array2)
ICC_tmp2 = round(ICC_tmp2, 2)
print("Normalization ICC:", ICC_tmp2)

## generate final invariant features

In [126]:
# Normalization Function
def normalize_column(column, Ng, formula_number):
    column = float(column)
    Ng = float(Ng)
    if formula_number == 1:
        return column * Ng
    elif formula_number == 2:
        return column / Ng
    elif formula_number == 3:
        return column / (Ng * Ng)
    elif formula_number == 4:
        return column / (Ng * Ng * Ng)
    elif formula_number == 5:
        return column / (math.log(Ng))
    elif formula_number == 6:
        return column * (math.log(Ng))
    elif formula_number == 7:
        return column / (math.log(Ng * Ng))
    elif formula_number == 8:
        return column * Ng * Ng
    elif formula_number == 9:
        return column * Ng * Ng * Ng
    else:
        raise ValueError(f"Invalid formula number: {formula_number}")


def calculate_icc(data):
    return icc(data.astype(float), icc_type="ICC(3,1)")

def find_best_normalization_method(df, ng_values):
    best_method = None  
    best_icc = -float("inf")  
    for formula_number in range(1, 10): 
        df_norm = df.copy()
        
        for column in ng_values:
            df_norm[column] = df[column].astype(float).apply(lambda x: normalize_column(x, float(column), formula_number))
        
        array = df_norm.values.astype(float)

        icc_value = calculate_icc(array)
        if icc_value > best_icc:
            best_icc = icc_value
            best_method = formula_number  
    return best_method

def generate_best_methods_sheet(input_path, output_path):

    features_file = pd.ExcelFile(input_path)
    sheet_names = features_file.sheet_names  
    best_methods = {}

    for sheet_name in sheet_names:

        df_feature = features_file.parse(sheet_name)
        ng_values = df_feature.columns.tolist()
        
        best_method = find_best_normalization_method(df_feature, ng_values)
        
        best_methods[sheet_name] = best_method

    best_methods_df = pd.DataFrame.from_dict(best_methods, orient="index", columns=["Best Method"])
    best_methods_df.index.name = "Feature"

    best_methods_df.to_excel(output_path, index=True)


In [ ]:
# inphase
input_excel_path = './Invariant_Features/inphase_feature_vs_Ng.xlsx'
output_excel_path = './Invariant_Features/Inphase_Best_Normalization_Methods.xlsx'
generate_best_methods_sheet(input_excel_path, output_excel_path)